IMPORTING MODULES


In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
import string
import nltk
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')

pyo.init_notebook_mode()
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA
from wordcloud import WordCloud,STOPWORDS

plt.rc('figure',figsize=(17,13))
import plotly.express as px
import plotly.graph_objs as go
import plotly.offline as pyo
from plotly.subplots import make_subplots


In [ ]:
!pip install vaderSentiment
!pip install twython

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# ****Pfizer and BioNTech Vaccine Tweets Analysis****

My first notebook, comments, suggestions  and upvotes are all welcome :)

In [ ]:
data=pd.read_csv('../input/pfizer-vaccine-tweets/vaccination_tweets.csv')
data.head()

# **TEXT PREPROCESSING FOR VADER SENTIMENT ANALYSIS**
* Here I didn't touch punctuations and case for now as VADER SENTIMENT ANALYSIS SCORES are affected by factors like punctuations,capitalization,preceeding-trigrams,degree modifiers,conjunctions etc.
* The Sentiments have been classified based on scores according to the VADER convention.
* A new column 'Sentiment'is thus, added giving sentiment corresponding to each Tweet text.

In [ ]:
def clean(text):
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub(r'\s+', ' ', text, flags=re.I)
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = re.sub('<.*?>+', '', text)
    return text
    


data['text'] = data['text'].apply(lambda x:clean(x))
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()
scores=[]
for i in range(len(data['text'])):
    
    score = analyser.polarity_scores(data['text'][i])
    score=score['compound']
    scores.append(score)
sentiment=[]
for i in scores:
    if i>=0.05:
        sentiment.append('Positive')
    elif i<=(-0.05):
        sentiment.append('Negative')
    else:
        sentiment.append('Neutral')
data['sentiment']=pd.Series(np.array(sentiment))

In [ ]:
data.head()

# **WORDCLOUD ANALYSIS OF TWEET TEXT**

In [ ]:
from wordcloud import WordCloud, STOPWORDS , ImageColorGenerator

tweet_All = " ".join(review for review in data.text)


fig, ax = plt.subplots(1, 1, figsize  = (10,10))
# Create and generate a word cloud image:
wordcloud_ALL = WordCloud(max_font_size=50, max_words=100, background_color="black").generate(tweet_All)

# Display the generated image:
ax.imshow(wordcloud_ALL, interpolation='bilinear')

ax.axis('off')


 **REMOVING PUNCTUATIONS AND MAKING TEXT LOWERCASE**

In [ ]:
def clean_text(text):
    
    text = str(text).lower()
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    
    return text
data['text'] = data['text'].apply(lambda x:clean_text(x))

In [ ]:
data['text']

#  **STEMMING AND LEMMATIZATION**

* df shows the text at each step of the preprocessing
* From the dataframe df we can decide which one of the stemming(Porter or Snowball)/lemmatization or both is suitable for our data.


In [ ]:
df=pd.DataFrame()
df['text']=data['text']
def tokenization(text):
    text = re.split('\W+', text)
    return text

df['tokenized'] = df['text'].apply(lambda x: tokenization(x.lower()))
stopword = nltk.corpus.stopwords.words('english')
def remove_stopwords(text):
    text = [word for word in text if word not in stopword]
    return text
    
df['No_stopwords'] = df['tokenized'].apply(lambda x: remove_stopwords(x))

ps = nltk.PorterStemmer()

def stemming1(text):
    text = [ps.stem(word) for word in text]
    return text

df['stemmed_porter'] = df['No_stopwords'].apply(lambda x: stemming1(x))

from nltk.stem.snowball import SnowballStemmer
s_stemmer = SnowballStemmer(language='english')
def stemming2(text):
    text = [s_stemmer.stem(word) for word in text]
    return text
df['stemmed_snowball'] = df['No_stopwords'].apply(lambda x: stemming2(x))

wn = nltk.WordNetLemmatizer()

def lemmatizer(text):
    text = [wn.lemmatize(word) for word in text]
    return text

df['lemmatized'] = df['No_stopwords'].apply(lambda x: lemmatizer(x))

In [ ]:
df.head()

IT LOOKS LIKE BOTH THE STEMMING METHODS ARE REDUCING THE TEXT WORDS TO UNUSUAL STEMS WHILE LEMMATIZATION SEEMS TO WORK FINE 


In [ ]:
data['text']=df['lemmatized']
data.head()

# **EDA PART:**

# SENTIMENT COUNTS

In [ ]:
temp = data.groupby('sentiment').count()['text'].reset_index().sort_values(by='text',ascending=False)
temp.style.background_gradient(cmap='Purples')

# FUNNEL CHART FOR BETTER VISUALIZATION OF SENTIMENT DISTRIBUTION


In [ ]:

plt.figure(figsize=(12,6))
sns.countplot(x='sentiment',data=data)
fig = go.Figure(go.Funnelarea(
    text =temp.sentiment,
    values = temp.text,
    title = {"position": "top center", "text": "Funnel-Chart of Sentiment Distribution"}
    ))
fig.show()

# RANKING THE SENTIMENTS BASED ON THREE METRICS:
1. THE FIRST METRIC IS THE SPECIFIC SENTIMENT'S COUNT THROUGHOUT THE TWEET DATA.NEUTRAL TWEETS ARE AT TOP HERE.
2. THE SECOND METRIC IS THE SUM OF FAVORITES ON A PARTICULAR SENTIMENT'S TWEET.IT CAN BE SEEN THAT THE POSITIVE TWEETS GET THE HIGHEST FAVORITES EVEN THOUGH NEUTRAL TWEETS HAVE THE HIGHEST COUNT.ATLEAST PEOPLE WANT TO BE POSITIVE WHEN IT'S A VACCINE BEING TALKED ABOUT.
3. THE THIRD METRIC IS THE SUM OF RETWEETS ON A PARTICULAR SENTIMENT'S TWEET.IT CAN BE SEEN THAT THE NEUTRAL TWEETS GET THE HIGHEST RETWEETS 

NEGATIVE TWEETS IS AT THE LAST POSITION IN ALL THE THREE METRICS

In [ ]:
 
genre_difference_metric = [data['sentiment'].value_counts().index,data.groupby(['sentiment']).sum()['favorites'].sort_values(ascending=False).index,data.groupby(['sentiment']).sum()['retweets'].sort_values(ascending=False).index]

#Dataframe to be used for plotting.
genre_evolution_df = pd.DataFrame(columns=['genre', 'rank_type', 'rank'])

#Populate the dataframe
for metric in range(3):
    for genre in range(len(genre_difference_metric[metric])):
        genre_evolution_df = genre_evolution_df.append({'genre':genre_difference_metric[metric][genre], 'rank_type': metric, 'rank':genre},
                                   ignore_index=True)
plt.style.use('seaborn-bright')       
fig = plt.figure(figsize=(10,6))
ax = fig.add_subplot(111)

    

ax.set_xlim([-2,4])
xs = [0.0, 1.0, 2.0]
x_labels = ['sentiment count', 'sum of favorites', 'sum of retweets']
plt.xticks(range(len(xs)), x_labels)
plt.xticks(xs, x_labels, rotation='vertical')
sns.pointplot(x=genre_evolution_df.rank_type,
              y=3-genre_evolution_df['rank'], 
              hue=genre_evolution_df.genre)
ys = range(1,5)
y_labels = ['3rd', '2nd', '1st']
plt.yticks(ys, y_labels)
ax.set_ylabel('Sentiment Category Rank')

plt.legend(bbox_to_anchor=(0.7, 0., 0.5, 0.5),loc='best',ncol=1)
plt.show();  

# ACCOUNT:VERIFIED/NON-VERIFIED

In [ ]:

sns.catplot(data=data, x='user_verified', kind= 'count',height=5,aspect=2)


# DISTRIBUTION OF THE FAVORITES RECIEVED BASED ON TWEET SENTIMENT AND WHETHER THE ACCOUNT IS VERIFIED/NOT

In [ ]:
sns.barplot(x="user_verified", y="favorites", hue="sentiment", data=data)


# DISTRIBUTION OF THE RETWEETS BASED ON TWEET SENTIMENT AND WHETHER THE ACCOUNT IS VERIFIED/NOT

In [ ]:
sns.barplot(x="user_verified", y="retweets", hue="sentiment", data=data)

# **THE TOP 30 MOST FREQUENTLY OCCURING WORDS IN THE TWEET TEXT DATA**


In [ ]:
all_words=[]
for i in range(len(data['text'])):
    a=data['text'][i]
    for i in a:
        all_words.append(i)
all_words=pd.Series(np.array(all_words))

common_words=all_words.value_counts()[:30].rename_axis('Common Words').reset_index(name='count')

fig = px.treemap(common_words, path=['Common Words'], values='count',title='30 Most Common Words In Tweets')
fig.show()

# **THE TOP 30 MOST FREQUENTLY USED HASHTAGS IN THE TWEET TEXT DATA**

In [ ]:
data['hashtags']=data['hashtags'].fillna('[]')
all_hashtags=[]
for i in range(len(data['hashtags'])):
    a=data['hashtags'][i].strip('][').split(', ') 
    for i in a:
        all_hashtags.append(i)
all_hashtags=['No Hashtag' if x=='' else x for x in all_hashtags]       

all_hashtags=pd.Series(np.array(all_hashtags))
print('There are {} instances of tweets in which No Hashtags were used'.format(all_hashtags.value_counts()[1]))

common_hashtags=all_hashtags.value_counts().drop(labels='No Hashtag')[:30].rename_axis('Common Hashtags').reset_index(name='count')
fig = px.treemap(common_hashtags, path=['Common Hashtags'], values='count',title='30 Most Common Hashtags')
fig.show()

# **SENTIMENT WISE ANALYSIS OF HASHTAGS AND WORDS:**

In [ ]:
Positive_tweet = data[data['sentiment']=='Positive'].reset_index()
Negative_tweet = data[data['sentiment']=='Negative'].reset_index()
Neutral_tweet = data[data['sentiment']=='Neutral'].reset_index()

# POSITIVE TWEETS

In [ ]:
all_positive_words=[]
all_positive_hashtags=[]
for i in range(len(Positive_tweet['text'])):
    a=Positive_tweet['text'][i]
    b=Positive_tweet['hashtags'][i].strip('][').split(', ')
    for i in a:
        all_positive_words.append(i)
    for i in b:
        all_positive_hashtags.append(i)
all_positive_words=pd.Series(np.array(all_positive_words))
all_positive_hashtags=pd.Series(np.array(all_positive_hashtags))
common_words=all_positive_words.value_counts().drop(labels='')[:70].rename_axis('Common Positive Words').reset_index(name='count')
fig = px.treemap(common_words, path=['Common Positive Words'], values='count',title='70 Most Common Words In Positive Tweets')
fig.show()
common_hashtags=all_positive_hashtags.value_counts()[:70].drop(labels='').rename_axis('Common Positive Hashtags').reset_index(name='count')
fig = px.treemap(common_hashtags, path=['Common Positive Hashtags'], values='count',title='70 Most Common Hashtags In Positive Tweets')
fig.show()

# NEGATIVE TWEETS

In [ ]:
all_negative_words=[]
all_negative_hashtags=[]
for i in range(len(Negative_tweet['text'])):
    a=Negative_tweet['text'][i]
    b=Negative_tweet['hashtags'][i].strip('][').split(', ')
    for i in a:
        all_negative_words.append(i)
    for i in b:
        all_negative_hashtags.append(i)
all_negative_words=pd.Series(np.array(all_negative_words))
all_negative_hashtags=pd.Series(np.array(all_negative_hashtags))
common_words=all_negative_words.value_counts().drop(labels='')[:70].rename_axis('Common Negative Words').reset_index(name='count')
fig = px.treemap(common_words, path=['Common Negative Words'], values='count',title='70 Most Common Words In Negative Tweets')
fig.show()
common_hashtags=all_negative_hashtags.value_counts()[:70].drop(labels='').rename_axis('Common Negative Hashtags').reset_index(name='count')
fig = px.treemap(common_hashtags, path=['Common Negative Hashtags'], values='count',title='70 Most Common Hashtags In Negative Tweets')
fig.show()

# NEUTRAL TWEETS

In [ ]:
all_neutral_words=[]
all_neutral_hashtags=[]
for i in range(len(Neutral_tweet['text'])):
    a=Neutral_tweet['text'][i]
    b=Neutral_tweet['hashtags'][i].strip('][').split(', ')
    for i in a:
        all_neutral_words.append(i)
    for i in b:
        all_neutral_hashtags.append(i)
all_neutral_words=pd.Series(np.array(all_neutral_words))
all_neutral_hashtags=pd.Series(np.array(all_neutral_hashtags))
common_words=all_neutral_words.value_counts().drop(labels='')[:70].rename_axis('Common Neutral Words').reset_index(name='count')
fig = px.treemap(common_words, path=['Common Neutral Words'], values='count',title='70 Most Common Words In Neutral Tweets')
fig.show()
common_hashtags=all_neutral_hashtags.value_counts()[:70].drop(labels='').rename_axis('Common Neutral Hashtags').reset_index(name='count')
fig = px.treemap(common_hashtags, path=['Common Neutral Hashtags'], values='count',title='70 Most Common Hashtags In Neutral Tweets')
fig.show()

It can be seen that 'PfizerBioNTech','COVID19','vaccine',etc are common in all the sentiments and are not giving a clear distinction in the words used exclusively for a particular sentiment.
So,the following plots show the distribution of words that are unique to each sentiment.


In [ ]:
common=set(all_positive_words).intersection(set(all_negative_words)).intersection(set(all_neutral_words))
common_list=list(common)

common_words=all_negative_words.value_counts().drop(labels=common_list)[:30].rename_axis('Common Negative Words').reset_index(name='count')
fig = px.treemap(common_words, path=['Common Negative Words'], values='count',title='Top 30 Unique Words In Negative Tweets')
fig.show()
common_words=all_positive_words.value_counts().drop(labels=common_list)[:30].rename_axis('Common Positive Words').reset_index(name='count')
fig = px.treemap(common_words, path=['Common Positive Words'], values='count',title='Top 30 Unique Words In Positive Tweets')
fig.show()
common_words=all_neutral_words.value_counts().drop(labels=common_list)[:30].rename_axis('Common Neutral Words').reset_index(name='count')
fig = px.treemap(common_words, path=['Common Neutral Words'], values='count',title='Top 30 Unique Words In Neutral Tweets')
fig.show()

This gave a clearer and better analysis!

# SOURCE DISTRIBUTION OF TWEETS

In [ ]:
data_ = data['source'].value_counts().reset_index()

trace1 = go.Bar(
                x = ['Twitter for Android', 'Twitter Web App', 'Twitter for iPhone',
       'TweetDeck', 'Buffer', 'Twitter for iPad', 'Twitter Media Studio',
       'ThreadReaderApp', 'Instagram', 'SocialFlow', 'Hootsuite Inc.',
       'LinkedIn', 'Twitter for Mac', '24liveblog', 'Publer ', 'IFTTT',
       'Socialbakers', 'Falcon Social Media Management ', 'Echobox',
       'Microsoft Power Platform', 'Nonli', 'Sendible',
       'Tweetbot for Mac', 'EastMojo',
       'Twitter Media Studio - LiveCut'], #temp_df['index'],
                y = data_['source'],
                marker = dict(color = 'rgb(250,13,92)',
                              line=dict(color='rgb(0,0,0)',width=1.5)),
                text=data_['source'], textposition='outside')
layout = go.Layout(template= "plotly_dark",title = 'SOURCE DISTRIBUTION OF TWEETS' , xaxis = dict(title = 'SOURCE'), yaxis = dict(title = 'Count'), height=650)
fig = go.Figure(data = [trace1], layout = layout)
fig.show()

# GLIMPSE AT VERIFIED ACCOUNTS


# **SOURCE DISTRIBUTION**

In [ ]:
data_verified=data[(data['user_verified']==True)].reset_index()
data_not_verified=data[(data['user_verified']==False)].reset_index()

In [ ]:

data_ = data_verified['source'].value_counts().reset_index()

trace1 = go.Bar(
                x = ['Twitter Web App', 'Twitter for iPhone', 'TweetDeck', 'Buffer',
       'SocialFlow', 'Hootsuite Inc.', 'Twitter for iPad',
       'Twitter for Android', 'Socialbakers', 'Echobox',
       'Twitter Media Studio', 'EastMojo',
       'Twitter Media Studio - LiveCut', 'GT_Backend'], #temp_df['index'],
                y = data_['source'],
                marker = dict(color = 'rgb(250,13,92)',
                              line=dict(color='rgb(0,0,0)',width=1.5)),
                text=data_['source'], textposition='outside')
layout = go.Layout(template= "plotly_dark",title = 'SOURCE DISTRIBUTION OF TWEETS FROM VERIFIED ACCOUNTS' , xaxis = dict(title = 'SOURCE'), yaxis = dict(title = 'Count'), height=650)
fig = go.Figure(data = [trace1], layout = layout)
fig.show()

# **HASHTAGS**

In [ ]:
all_hashtags=[]
for i in range(len(data_verified['hashtags'])):
    a=data_verified['hashtags'][i].strip('][').split(', ') 
    for i in a:
        all_hashtags.append(i)
all_hashtags=['No Hashtag' if x=='' else x for x in all_hashtags]       

all_hashtags=pd.Series(np.array(all_hashtags))
common_hashtags=all_hashtags.value_counts()[:30].rename_axis('Common Hashtags').reset_index(name='count')
fig = px.treemap(common_hashtags, path=['Common Hashtags'], values='count',title='30 Most Common Hashtags by VERIFIED ACCOUNTS')
fig.show()

# **SENTIMENT WISE WORD FREQUENCY**

In [ ]:


Positive_tweet = data_verified[data_verified['sentiment']=='Positive'].reset_index()
Negative_tweet = data_verified[data_verified['sentiment']=='Negative'].reset_index()
Neutral_tweet = data_verified[data_verified['sentiment']=='Neutral'].reset_index()
all_positive_words=[]

for i in range(len(Positive_tweet['text'])):
    a=Positive_tweet['text'][i]
    
    for i in a:
        all_positive_words.append(i)
    
all_positive_words=pd.Series(np.array(all_positive_words))
all_neutral_words=[]

for i in range(len(Neutral_tweet['text'])):
    a=Neutral_tweet['text'][i]
    
    for i in a:
        all_neutral_words.append(i)
    
all_neutral_words=pd.Series(np.array(all_neutral_words))
all_negative_words=[]

for i in range(len(Negative_tweet['text'])):
    a=Negative_tweet['text'][i]
   
    for i in a:
        all_negative_words.append(i)
    
all_negative_words=pd.Series(np.array(all_negative_words))
common=set(all_positive_words).intersection(set(all_negative_words)).intersection(set(all_neutral_words))
common_list=list(common)

common_words=all_negative_words.value_counts().drop(labels=common_list)[:30].rename_axis('Common Negative Words').reset_index(name='count')
fig = px.treemap(common_words, path=['Common Negative Words'], values='count',title='30 Most Common Unique Negative Words by VERIFIED ACCOUNTS')
fig.show()
common_words=all_positive_words.value_counts().drop(labels=common_list)[:30].rename_axis('Common Positive Words').reset_index(name='count')
fig = px.treemap(common_words, path=['Common Positive Words'], values='count',title='30 Most Common Unique Positive Words by VERIFIED ACCOUNTS')
fig.show()
common_words=all_neutral_words.value_counts().drop(labels=common_list)[:30].rename_axis('Common Neutral Words').reset_index(name='count')
fig = px.treemap(common_words, path=['Common Neutral Words'], values='count',title='30 Most Common Unique Neutral Words by VERIFIED ACCOUNTS')
fig.show()

#  TWEET SENTIMENT BASED ANALYSIS OF LOCATION

In [ ]:
data['user_location'] = data['user_location'].fillna('NaN')
Positive_tweet = data[data['sentiment']=='Positive'].reset_index()
Negative_tweet = data[data['sentiment']=='Negative'].reset_index()
Neutral_tweet = data[data['sentiment']=='Neutral'].reset_index()
pos_location=Positive_tweet['user_location']
neg_location=Negative_tweet['user_location']
neu_location=Neutral_tweet['user_location']

common=set(pos_location).intersection(set(neg_location)).intersection(set(neu_location))
common_list=list(common)

common_words=neg_location.value_counts().drop(labels=common_list)[:10].rename_axis('Common Negative Locations').reset_index(name='count')
fig = px.treemap(common_words, path=['Common Negative Locations'], values='count',title='10 Top Unique Negative Tweets Locations')
fig.show()
common_words=pos_location.value_counts().drop(labels=common_list)[:10].rename_axis('Common Positive Locations').reset_index(name='count')
fig = px.treemap(common_words, path=['Common Positive Locations'], values='count',title='10 Top Unique Positive Tweets Locations')
fig.show()
common_words=neu_location.value_counts().drop(labels=common_list)[:10].rename_axis('Common Neutral Locations').reset_index(name='count')
fig = px.treemap(common_words, path=['Common Neutral Locations'], values='count',title='10 Top Unique Neutral Tweets Locations')
fig.show()

# 'ACCOUNTS WITH HIGHEST FOLLOWERS' ANALYSIS BASED ON THE TWEET SENTIMENT

In [ ]:
from matplotlib import rcParams


In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(3,1, figsize=(10, 16))
sns.barplot(x="user_followers", y="user_name", orient="h", ax=ax1, palette=["b"],
           data=data[(data.sentiment== "Positive")]\
           .drop_duplicates(subset=["user_name"])\
           .sort_values(by=["user_followers"], ascending=False)[["user_name", "user_followers"]][:10])
ax1.set_title('Top 10 Accounts with Highest Followers who tweet Positive')
sns.barplot(x="user_followers", y="user_name", orient="h", ax=ax2, palette=["g"],
           data=data[(data.sentiment == "Neutral")]
           .drop_duplicates(subset=["user_name"])\
           .sort_values(by=["user_followers"], ascending=False)[["user_name", "user_followers"]][:10])
ax2.set_title('Top 10 Accounts with Highest Followers who tweet Neutral')
sns.barplot(x="user_followers", y="user_name", orient="h", ax=ax3, palette=["r"],
           data=data[(data.sentiment == "Negative")]
           .drop_duplicates(subset=["user_name"])\
           .sort_values(by=["user_followers"], ascending=False)[["user_name", "user_followers"]][:10])
ax3.set_title('Top 10 Accounts with Highest Followers who tweet Negative')

fig.show()

# TIME BASED ANALYSIS OF TWEET SENTIMENT:


* A sudden fall in the positive tweets can be seen around 22-25 Dec.
* A sudden spike in negative tweets can be seen around 30 Dec 2020.

In [ ]:
data["date"] = pd.to_datetime(data.date) 
timeline = data.resample('D', on='date')["sentiment"].value_counts().unstack(1)

timeline.reset_index(inplace=True)

timeline = timeline.melt("date", var_name='sentiment',  value_name='vals')

sns.set_style("whitegrid")
sns.lineplot(x="date", y="vals", hue="sentiment", data=timeline, palette=["r", "g","b"])
plt.figure(figsize=(40,10))

**THANKYOU FOR READING MY FIRST NOTEBOOK.
DO UPVOTE AND GIVE FEEDBACK IF YOU FOUND IT USEFUL!**